<a href="https://colab.research.google.com/github/shakinul-islam/Python/blob/main/NLP_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas scikit-learn emoji beautifulsoup4

In [4]:
from google.colab import files
uploaded = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset.csv


#how TF IDF works (theory behind it)
#refgression
#logistic regression
#use hugging face transformer for both tokenization and classification on IMBD dataset

In [6]:


import pandas as pd
import re
from emoji import demojize
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 1: Load IMDB dataset
df = pd.read_csv('IMDB Dataset.csv')

# Step 2: Map sentiment to 0/1
df['label'] = df['sentiment'].map({'positive':1, 'negative':0})

# Step 3: Preprocessing function
def clean_review(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # remove HTML
    text = demojize(text)  # emoji to text
    text = text.lower()    # lowercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # remove special chars
    return text

# Step 4: Apply preprocessing
df['cleaned_review'] = df['review'].apply(clean_review)

# Step 5: Display before/after for first 5 reviews
print("=== Sample Before/After Cleaning ===")
for i in range(5):
    sentiment = "Positive" if df['label'][i]==1 else "Negative"
    print(f"Original Review {i+1} ({sentiment}):\n{df['review'][i]}\n")
    print(f"Cleaned Review {i+1}:\n{df['cleaned_review'][i]}\n")
    print("="*80)


# Step 6: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_review'], df['label'], test_size=0.2, random_state=42
)

# Step 7: Text Vectorization (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Step 8: Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Step 9: Predict & Accuracy
y_pred = model.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {acc*100:.2f}%")

# Step 10: Test on sample review
sample = ["I love this movie! 😍", "This movie is terrible... 😡"]
sample_cleaned = [clean_review(r) for r in sample]
sample_vec = vectorizer.transform(sample_cleaned)
predictions = model.predict(sample_vec)
for r, p in zip(sample, predictions):
    label = "Positive" if p==1 else "Negative"
    print(f"Review: {r} --> Predicted Sentiment: {label}")


=== Sample Before/After Cleaning ===
Original Review 1 (Positive):
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<b